In [1]:
%%bash
cd /core
git config remote.origin.fetch "+refs/heads/*:refs/remotes/origin/*" && git fetch && git checkout main
pip install . --no-deps

cd /dataloader
git config remote.origin.fetch "+refs/heads/*:refs/remotes/origin/*" && git fetch && git checkout main
pip install . --no-deps

cd /nvtabular
git config remote.origin.fetch "+refs/heads/*:refs/remotes/origin/*" && git fetch && git checkout main
pip install . --no-deps

cd /systems
git config remote.origin.fetch "+refs/heads/*:refs/remotes/origin/*" && git fetch && git checkout main
pip install . --no-deps

cd /transformers4rec
git config remote.origin.fetch "+refs/heads/*:refs/remotes/origin/*" && git fetch && git checkout main
pip install . --no-deps

fatal: the remote end hung up unexpectedly
fatal: early EOF
fatal: index-pack failed


Processing /core
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for merlin-core: filename=merlin_core-23.5.0+13.gd6720139-py3-none-any.whl size=171826 sha256=15d38758bf2b2832c7d57587004e22a93ed40d5a517fc426bea34bac5a8382a4
  Stored in directory: /tmp/pip-ephem-wheel-cache-_n78fkiv/wheels/8f/da/8c/c779661788874afaa32fd10abeac6016635956e3bad9940584
Successfully built merlin-core
  Attempting uninstall: merlin-core
    Found existing installation: merlin-core 23.5.0+13.gd6720139
    Uninstalling merlin-core-23.5.0+13.gd6720139:
      Successfully uninstalled merlin-core-23.5.0+13.gd6720139


From https://github.com/NVIDIA-Merlin/dataloader
   2e05300..6d599d5  main       -> origin/main
   5463de1..64b31ae  gh-pages   -> origin/gh-pages
Already on 'main'


Your branch is behind 'origin/main' by 1 commit, and can be fast-forwarded.
  (use "git pull" to update your local branch)
Processing /dataloader
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for merlin-dataloader: filename=merlin_dataloader-23.5.0+2.g2e05300-py3-none-any.whl size=35632 sha256=c1844f9cb19138d3f68fccd9ad4f803344976784b4acb9d32cc154f95ecaea6d
  Stored in directory: /tmp/pip-ephem-wheel-cache-8vlio1b_/wheels/8c/19/5b/15dc04f5a977f6a7f73ed66c91996a687b1d9e3154a4765536
Successfully built merlin-dataloader
  Attempting uninstall: merlin-dataloader
    Found existing installation: merlin-dataloader 23.5.0+2.g2e05300
    Uninstalling merlin-dataloader-23.5.0+2.g2e05300:
      Successfully uninstalled

From https://github.com/NVIDIA-Merlin/NVTabular
   25151f73..d26e776a  main              -> origin/main
   9da40a0c..a9b0014e  gh-pages          -> origin/gh-pages
 * [new branch]        pull-request/1843 -> origin/pull-request/1843
Already on 'main'


Your branch is behind 'origin/main' by 1 commit, and can be fast-forwarded.
  (use "git pull" to update your local branch)
Processing /nvtabular
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for nvtabular: filename=nvtabular-23.5.0+16.g25151f73-cp38-cp38-linux_x86_64.whl size=241878 sha256=2b237a670e185d93e20a73fe3ab70e6885838612ce32ac1013ab5bcfa9a2fc30
  Stored in directory: /tmp/pip-ephem-wheel-cache-v_jr5e61/wheels/df/bf/c2/9cc2a62fe6da42038c26a9c0c4e25f9767093528b102fa30a2
Successfully built nvtabular
  Attempting uninstall: nvtabular
    Found existing installation: nvtabular 23.5.0+16.g25151f73
    Uninstalling nvtabular-23.5.0+16.g25151f73:
      Successfully uninstalled nvtabular-23.5.0+16.g25151f73


From https://github.com/NVIDIA-Merlin/systems
   b23ff21..0d8ca65  main             -> origin/main
   344a74a..518a990  gh-pages         -> origin/gh-pages
 * [new branch]      pull-request/345 -> origin/pull-request/345
   dccb618..6b3bf41  pull-request/349 -> origin/pull-request/349
Already on 'main'


Your branch is behind 'origin/main' by 7 commits, and can be fast-forwarded.
  (use "git pull" to update your local branch)
Processing /systems
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for merlin-systems: filename=merlin_systems-23.5.0+9.gb23ff21-py3-none-any.whl size=82964 sha256=3111fe71c39bf67d98b885ebb524ed66d9b609e988033638bc577e8f80978dfa
  Stored in directory: /tmp/pip-ephem-wheel-cache-pom6090e/wheels/1f/e9/71/1b0c6295aa7f4b37cb70292d96d87d9f38204674e6531bdda6
Successfully built merlin-systems
  Attempting uninstall: merlin-systems
    Found existing installation: merlin-systems 23.5.0+9.gb23ff21
    Uninstalling merlin-systems-23.5.0+9.gb23ff21:
      Successfully uninstalled merlin-systems-23.5

From https://github.com/NVIDIA-Merlin/Transformers4Rec
   6a7c9746..bfcaa303  gh-pages          -> origin/gh-pages
 * [new branch]        pull-request/720  -> origin/pull-request/720
 * [new branch]        recall_metric_fix -> origin/recall_metric_fix
Already on 'main'


Your branch is up to date with 'origin/main'.
Processing /transformers4rec
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for transformers4rec: filename=transformers4rec-23.5.0+9.gf4946bfa-py3-none-any.whl size=488200 sha256=ce1d93b4079d4fe20cb54a7389fbfcdc71fa262ca1a1a86013cc1e7da7e6219f
  Stored in directory: /tmp/pip-ephem-wheel-cache-2xydu6e0/wheels/24/44/e3/c29f7de8e7315585705f880ad32ffeae66fcaeb79003405ef6
Successfully built transformers4rec
  Attempting uninstall: transformers4rec
    Found existing installation: transformers4rec 23.5.0+9.gf4946bfa
    Uninstalling transformers4rec-23.5.0+9.gf4946bfa:
      Successfully uninstalled transformers4rec-23.5.0+9.gf4946bfa


In [2]:
# Copyright 2022 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions anda
# limitations under the License.
# ==============================================================================

# Each user is responsible for checking the content of datasets and the
# applicable licenses and determining if suitable for the intended use.

<img src="https://developer.download.nvidia.com/notebooks/dlsw-notebooks/merlin_models-transformers-net-item-prediction/nvidia_logo.png" style="width: 90px; float: right;">

# Transformer-based architecture for next-item prediction task with pretrained embeddings

This notebook is created using the latest stable [merlin-tensorflow](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/merlin/containers/merlin-tensorflow/tags) container.

## Overview

In this use case we will train a Transformer-based architecture for next-item prediction task with pretrained embeddings.

**You can chose to download the full dataset manually or use synthetic data.**

We will use the [SIGIR eCOM 2021 Data Challenge Dataset](https://github.com/coveooss/SIGIR-ecom-data-challenge) to train a session-based model. The dataset contains 36M events of users browsing an online store.

We will reshape the data to organize it into 'sessions'. Each session will be a full customer online journey in chronological order. The goal will be to predict the `url` of the next action taken.


### Learning objectives

- Training a Transformer-based architecture for next-item prediction task

## Downloading and preparing the dataset

In [3]:
import os
import cudf
import numpy as np
import pandas as pd
import nvtabular as nvt
from merlin.schema import ColumnSchema, Schema, Tags

OUTPUT_DATA_DIR = os.environ.get('OUTPUT_DATA_DIR', '/workspace/data')

2023-06-13 21:44:34.072912: I tensorflow/core/platform/cpu_feature_guard.cc:194] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/lib/python3.8/dist-packages/merlin/dtypes/mappings/torch.py:43: UserWarning: PyTorch dtype mappings did not load successfully due to an error: No module named 'torch'
  warn(f"PyTorch dtype mappings did not load successfully due to an error: {exc.msg}")
2023-06-13 21:44:35.595381: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-13 21:44:35.595729: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had nega

You can download the full dataset by registering [here](https://www.coveo.com/en/ailabs/sigir-ecom-data-challenge). If you chose to download the data, please place it alongside this notebook in the `sigir_dataset` directory and extract it.

To process the downloaded data uncomment the cell below.

By default, in this notebook, we will be using synthetically generated data based on the SIGIR dataset.

In [4]:
# # Unocomment this cell to use the original SIGIR dataset.

# train = nvt.Dataset('/workspace/sigir_dataset/train/browsing_train.csv', part_size='500MB')
# skus = nvt.Dataset('/workspace/sigir_dataset/train/sku_to_content.csv')

# skus = pd.read_csv('/workspace/sigir_dataset/train/sku_to_content.csv')

# skus['description_vector'] = skus['description_vector'].replace(np.nan, '')
# skus['image_vector'] = skus['image_vector'].replace(np.nan, '')

# skus['description_vector'] = skus['description_vector'].apply(lambda x: [] if len(x) == 0 else eval(x))
# skus['image_vector'] = skus['image_vector'].apply(lambda x: [] if len(x) == 0 else eval(x))
# skus = skus[skus.description_vector.apply(len) > 0]
# skus = nvt.Dataset(skus)

In [5]:
%%bash

cd /workspace && pip install .

Processing /workspace
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'


  Created wheel for merlin-models: filename=merlin_models-23.5.dev0+64.g6f459051.dirty-py3-none-any.whl size=384115 sha256=8b1a1fbcd316aadef9a5bcfaa1fe0b481a7b8dfc5d824c1300b11e8706653291
  Stored in directory: /tmp/pip-ephem-wheel-cache-flj_rnyy/wheels/59/14/70/d94958f41745fe226f3bc60bb3cabbbc8a98e4d6679e91038a
Successfully built merlin-models


ERROR: transformers4rec 23.5.0+9.gf4946bfa requires torchmetrics>=0.10.0, which is not installed.
ERROR: dask-cudf 22.12.0 requires cupy-cuda12x, which is not installed.
ERROR: cudf 22.12.0 requires cubinlinker, which is not installed.
ERROR: cudf 22.12.0 requires cupy-cuda12x, which is not installed.
ERROR: cudf 22.12.0 requires ptxcompiler, which is not installed.
ERROR: dask-cudf 22.12.0 has requirement dask==2022.11.1, but you'll have dask 2023.1.1 which is incompatible.
ERROR: dask-cudf 22.12.0 has requirement distributed==2022.11.1, but you'll have distributed 2023.1.1 which is incompatible.
ERROR: dask-cuda 22.12.0 has requirement dask==2022.11.1, but you'll have dask 2023.1.1 which is incompatible.
ERROR: dask-cuda 22.12.0 has requirement distributed==2022.11.1, but you'll have distributed 2023.1.1 which is incompatible.


  Attempting uninstall: merlin-models
    Found existing installation: merlin-models 23.5.dev0+45.g1a80f77e
    Uninstalling merlin-models-23.5.dev0+45.g1a80f77e:
      Successfully uninstalled merlin-models-23.5.dev0+45.g1a80f77e
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2022.5.0
    Uninstalling fsspec-2022.5.0:
      Successfully uninstalled fsspec-2022.5.0


In [6]:
from merlin.datasets.synthetic import generate_data

train = generate_data('sigir-browsing', 1000)
skus = generate_data('sigir-sku', 1000)

The `skus` dataset contains the mapping between the `product_sku_hash` (essentially an item id) to the `description_vector` -- an embedding obtained from the description.

To use this information in our model, we need to map the `product_sku_hash` information to an id.

But we need to make sure that the way we process `skus` and the `train` dataset (event information) is consistent. That the same `product_sku_hash` is mapped to the same id both when processing `skus` and `train`.

We do so by defining and fitting a `Categorify` op and using it to process both datasets.

In [7]:
cat_op = nvt.ops.Categorify()
out = ['product_sku_hash'] >> cat_op >> nvt.ops.TagAsItemID()
out += ['event_type', 'product_action', 'session_id_hash', 'hashed_url'] >> nvt.ops.Categorify()
out += ['server_timestamp_epoch_ms'] >> nvt.ops.NormalizeMinMax()

wf = nvt.Workflow(out)

train = wf.fit_transform(train)

train.head()

,product_sku_hash,event_type,product_action,session_id_hash,hashed_url,server_timestamp_epoch_ms
0,140,4,5,9,242,0.675520
1,307,3,3,17,283,0.736323
2,26,4,6,3,14,0.899771
3,548,3,3,125,488,0.944583
4,512,4,5,11,579,0.921439


Now that we have processed the train set, we can use the mapping preserved in the `cat_op` to process the `skus` dataset containing the embeddings we are after.

Let's now `Categorify` the `product_sku_hash` in `skus` and grab just the description embedding information.

In [8]:
skus.head()

,product_sku_hash,description_vector,category_hash,price_bucket
0,42,"[0.2593153276239712, 0.4894138440490076, -0.18...",105,0.716184
1,8,"[0.0950188853767045, 0.05981399327130871, -0.1...",92,0.736832
2,40,"[-0.1312345336472498, 0.5513486112044199, 0.35...",39,0.504082
3,4,"[0.28111895053468666, 0.36063517509116244, -0....",10,0.486001
4,42,"[0.11352680145797639, -0.41496256664223774, 0....",155,0.597133


In [9]:
out = ['product_sku_hash'] >> cat_op
wf = nvt.Workflow(out + 'description_vector')
skus_ds = wf.transform(skus)

skus_ds.head()

,product_sku_hash,description_vector
0,2,"[0.2593153276239712, 0.4894138440490076, -0.18..."
1,2,"[0.0950188853767045, 0.05981399327130871, -0.1..."
2,2,"[-0.1312345336472498, 0.5513486112044199, 0.35..."
3,81,"[0.28111895053468666, 0.36063517509116244, -0...."
4,2,"[0.11352680145797639, -0.41496256664223774, 0...."


Let us now export the embedding information to a `numpy` array and write it to disk.

We will later pass this information so that the `Loader` will load the correct emebedding for the products corresponding to the given step of a customer journey.

The embeddings are linked to the train set using the `product_sku_hash` information.

In [10]:
skus_ds.to_npy('skus.npy')

How will the `Loader` know which embedding to associated with a given row of the train set?

The `product_sku_hash` ids have been exported along with the embeddings and are contained in the first column of the output `numpy` array.

Here is the id of the first embedding stored in `skus.npy`.

In [11]:
np.load('skus.npy')[0, 0]

2.0

and here is the embedding vector corresponding to `product_sku_hash` of id referenced above:

In [12]:
np.load('skus.npy')[0, 1:]

array([ 0.25931533,  0.48941384, -0.18085134, -0.1474341 ,  0.40550731,
        0.55873592, -0.27944447,  0.25166949,  0.40163509, -0.27438004,
        0.10215773, -0.32745561,  0.57700774, -0.40935597,  0.11755104,
        0.01219579, -0.41041086,  0.3981805 , -0.3396262 ,  0.09288609,
       -0.2548211 ,  0.35589106,  0.43995202,  0.2900171 , -0.06615147,
        0.22055131,  0.16407797,  0.47078718,  0.25719827, -0.16493549,
        0.32950932, -0.07959482, -0.2713532 ,  0.53515961, -0.12554064,
       -0.21047772, -0.24527885, -0.15110281,  0.12987542, -0.01997079,
        0.45479069,  0.11293635, -0.27765685,  0.42883531, -0.33491447,
       -0.27391345,  0.02839783, -0.42921917,  0.33905582, -0.03977705])

Let us now construct the `Loader` that will provide the data to our model.

Let us first rearrange the `train` dataset to group the actions by `session_id_hash`. Actions within a session will be contained in a single row.

In [13]:
groupby_features = train.head().columns.tolist() >> nvt.ops.Groupby(
    groupby_cols=['session_id_hash'],
    aggs={
        'product_sku_hash': ['list'],
        'event_type': ['list'],
        'product_action': ['list'],
        'hashed_url': ['list', 'count'],
        'server_timestamp_epoch_ms': ['list']
    },
    sort_cols="server_timestamp_epoch_ms"
)

MINIMUM_SESSION_LENGTH = 5
filtered_sessions = groupby_features >> nvt.ops.Filter(f=lambda df: df["hashed_url_count"] >= MINIMUM_SESSION_LENGTH)

# We won't be needing the `session_id_hash` nor the `hashed_url_count` any longer
wf = nvt.Workflow(
    filtered_sessions[
        'product_sku_hash_list',
        'event_type_list',
        'product_action_list',
        'hashed_url_list',
    ]
)
train_processed = wf.fit_transform(train)

train_processed.head()

,product_sku_hash_list,event_type_list,product_action_list,hashed_url_list
0,"[134, 74, 414, 203, 509, 151, 189, 230, 63, 41...","[3, 4, 4, 3, 3, 4, 3, 4, 4, 4, 3, 4, 4, 4, 3, ...","[3, 4, 3, 3, 6, 6, 4, 3, 4, 3, 3, 4, 6, 5, 3, ...","[4, 259, 208, 80, 239, 330, 116, 28, 479, 18, ..."
1,"[291, 126, 34, 198, 325, 32, 127, 214, 30, 55,...","[4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 4, 3, 4, 3, 4, ...","[3, 3, 5, 3, 3, 4, 5, 5, 3, 4, 6, 3, 5, 3, 3, ...","[58, 503, 11, 205, 58, 212, 77, 604, 99, 24, 5..."
2,"[396, 72, 559, 7, 571, 40, 161, 619, 528, 30, ...","[3, 4, 3, 3, 4, 3, 3, 3, 3, 3, 3, 4, 4, 3, 4, ...","[4, 6, 3, 3, 4, 3, 5, 4, 4, 4, 6, 5, 4, 4, 5, ...","[549, 282, 192, 254, 517, 226, 249, 26, 168, 2..."
3,"[131, 148, 246, 72, 213, 416, 197, 249, 115, 2...","[3, 3, 3, 3, 4, 3, 4, 3, 4, 3, 3, 4, 4, 3, 4, ...","[5, 4, 3, 4, 6, 5, 5, 6, 4, 6, 5, 5, 3, 3, 6, ...","[54, 90, 19, 193, 34, 67, 560, 5, 185, 76, 62,..."
4,"[306, 5, 204, 465, 103, 92, 205, 49, 154, 562,...","[3, 3, 4, 3, 3, 4, 3, 3, 3, 3, 4, 3, 4, 3, 3, ...","[4, 5, 6, 4, 3, 6, 6, 5, 3, 6, 3, 5, 5, 5, 6, ...","[88, 110, 228, 420, 419, 204, 174, 14, 224, 69..."


Let us save the workflow -- we will load it later on to serve the trained model using the Triton Inference Server.

In [14]:
wf.save('data_preprocessing_workflow')

We are now ready to construct the `Loader` that will feed the data to our model.

We begin by reading in the embeddings information.

In [15]:
embeddings = np.load('skus.npy')

We are now ready to define the `Loader`.

In [16]:
from merlin.dataloader.tensorflow import Loader
from merlin.dataloader.ops.embeddings import EmbeddingOperator
import merlin.models.tf as mm

loader = Loader(
    train_processed,
    batch_size=10,
    transforms=[
        EmbeddingOperator(
            embeddings[:, 1:],
            id_lookup_table=embeddings[:, 0].astype(int),
            lookup_key="product_sku_hash_list",
        )
    ],
    shuffle=True
)

[INFO]: sparse_operation_kit is imported
[SOK INFO] Import /usr/local/lib/python3.8/dist-packages/merlin_sok-1.1.4-py3.8-linux-x86_64.egg/sparse_operation_kit/lib/libsok_experiment.so
[SOK INFO] Import /usr/local/lib/python3.8/dist-packages/merlin_sok-1.1.4-py3.8-linux-x86_64.egg/sparse_operation_kit/lib/libsok_experiment.so
[SOK INFO] Initialize finished, communication tool: horovod


2023-06-13 21:45:54.507105: I tensorflow/core/platform/cpu_feature_guard.cc:194] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-13 21:45:54.507961: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-13 21:45:54.508174: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-13 21:45:54.508333: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must 

Using the `EmbeddingOperator` object we referenced our `embeddings` and advised the model what to use as a key to look up the information.

Below is an example batch of data that our model will consume.

In [17]:
batch = mm.sample_batch(loader, batch_size=10, include_targets=False, prepare_features=True)
batch

{'product_sku_hash_list': <tf.RaggedTensor [[[412],
   [52],
   [152],
   [504],
   [105],
   [260],
   [77],
   [202],
   [16],
   [452],
   [7],
   [551],
   [326],
   [315],
   [129],
   [113],
   [413],
   [39],
   [317],
   [429]], [[22],
            [293],
            [136],
            [264],
            [130],
            [354],
            [58],
            [193],
            [500],
            [254],
            [114],
            [471],
            [445],
            [614],
            [552]], [[322],
                     [35],
                     [221],
                     [172],
                     [167],
                     [38],
                     [248],
                     [582],
                     [360]], [[526],
                              [267],
                              [519],
                              [505],
                              [272],
                              [85],
                              [196],
                              

## Creating and training the model

We are now ready to construct our model.

In [18]:
import merlin.models.tf as mm

input_block = mm.InputBlockV2(
    loader.output_schema,
    embeddings=mm.Embeddings(
        loader.output_schema.select_by_tag(Tags.CATEGORICAL),
        sequence_combiner=None,
    ),
    pretrained_embeddings=mm.PretrainedEmbeddings(
        loader.output_schema.select_by_tag(Tags.EMBEDDING),
        sequence_combiner=None,
        normalizer="l2-norm",
        output_dims={"embeddings": 128},
    )
)

We have now constructed an `input_block` that will take our batch and transform it in a fashion that will make it amenable for further processing by subsequent layers of our model.

To test that everything has worked, we can pass our exmple `batch` through the `input_block`.

In [19]:
input_batch = input_block(batch)
input_batch

<tf.RaggedTensor [[[0.0, 0.0, 0.0, ..., 0.021004785, 0.029854227, -0.0015123375],
  [0.0, 0.0, 0.0, ..., 0.040132772, -0.036564004, -0.0024443977],
  [0.0, 0.0, 0.0, ..., 0.01408584, 0.018433187, 0.0469519],
  ...,
  [0.0, 0.0, 0.0, ..., 0.039661456, 0.040508214, 0.03286426],
  [0.0, 0.0, 0.0, ..., -0.037068926, 0.016435985, -0.008453239],
  [0.0, 0.0, 0.0, ..., 0.045997214, 0.010021627, -0.01330347]]   ,
 [[0.0, 0.0, 0.0, ..., -0.018446874, 0.0069144852, 0.010627259],
  [-0.042437136, 0.048168633, -0.10456142, ..., -0.04735918, 0.033852246,
   0.028094146],
  [0.0, 0.0, 0.0, ..., 0.030445721, 0.009619497, -0.007990874],
  ...,
  [0.0, 0.0, 0.0, ..., -0.0146845095, 0.010920249, 0.04464158],
  [0.0, 0.0, 0.0, ..., -0.04935621, -0.0067207105, 0.010152161],
  [0.0, 0.0, 0.0, ..., 0.020124007, 0.032709096, -0.031755794]]          ,
 [[-0.031026319, -0.17222269, 0.076006435, ..., 0.0040291436, -0.02539729,
   -0.017466068],
  [0.0, 0.0, 0.0, ..., -0.02963065, -0.02561524, 0.013736669],
  [0

Let us now construct the remaining layers of our model.

In [20]:
target = 'hashed_url_list'

dmodel=128
mlp_block = mm.MLPBlock(
                [128,dmodel],
                activation='relu',
                no_activation_last_layer=True,
            )
transformer_block = mm.XLNetBlock(d_model=dmodel, n_head=4, n_layer=2)
model = mm.Model(
    input_block,
    mlp_block,
    transformer_block,
    mm.CategoricalOutput(
        train_processed.schema.select_by_name(target),
        default_loss="categorical_crossentropy",
    ),
)

And let us train it.

In [21]:
model.compile(run_eagerly=False, optimizer='adam', loss="categorical_crossentropy")
model.fit(loader, batch_size=64, epochs=5, pre=mm.SequenceMaskRandom(schema=loader.output_schema, target=target, masking_prob=0.3, transformer=transformer_block))

/usr/local/lib/python3.8/dist-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer TruncatedNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


Epoch 1/5


2023-06-13 21:46:02.256130: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8700


2023-06-13 21:46:12.754303: W tensorflow/core/grappler/optimizers/loop_optimizer.cc:907] Skipping loop optimization for Merge node with control input: model/xl_net_block/sequential_block_7/replace_masked_embeddings/RaggedWhere/Assert/AssertGuard/branch_executed/_95


6/6 [==============================] - 11s 22ms/step - loss: 6.7031 - recall_at_10: 0.0071 - mrr_at_10: 0.0012 - ndcg_at_10: 0.0025 - map_at_10: 0.0012 - precision_at_10: 7.0671e-04 - regularization_loss: 0.0000e+00 - loss_batch: 6.7274
Epoch 2/5
6/6 [==============================] - 0s 20ms/step - loss: 6.5760 - recall_at_10: 0.0263 - mrr_at_10: 0.0049 - ndcg_at_10: 0.0098 - map_at_10: 0.0049 - precision_at_10: 0.0026 - regularization_loss: 0.0000e+00 - loss_batch: 6.5537
Epoch 3/5
6/6 [==============================] - 0s 22ms/step - loss: 6.5651 - recall_at_10: 0.0294 - mrr_at_10: 0.0086 - ndcg_at_10: 0.0133 - map_at_10: 0.0086 - precision_at_10: 0.0029 - regularization_loss: 0.0000e+00 - loss_batch: 6.5210
Epoch 4/5
6/6 [==============================] - 0s 22ms/step - loss: 6.6248 - recall_at_10: 0.0415 - mrr_at_10: 0.0072 - ndcg_at_10: 0.0148 - map_at_10: 0.0072 - precision_at_10: 0.0042 - regularization_loss: 0.0000e+00 - loss_batch: 6.5936
Epoch 5/5
6/6 [======================

Let's save the model so that we can use it for serving.

In [22]:
model.save('trained_model')

  (_feature_shapes): Dict(
    (product_sku_hash_list): TensorShape([10, None, 1])
    (event_type_list): TensorShape([10, None, 1])
    (product_action_list): TensorShape([10, None, 1])
    (hashed_url_list): TensorShape([10, None, 1])
    (embeddings): TensorShape([10, None, 50])
  )
  (_feature_dtypes): Dict(
    (product_sku_hash_list): tf.int64
    (event_type_list): tf.int64
    (product_action_list): tf.int64
    (hashed_url_list): tf.int64
    (embeddings): tf.float32
  )
), because it is not built.
  (_feature_shapes): Dict(
    (product_sku_hash_list): TensorShape([10, None, 1])
    (event_type_list): TensorShape([10, None, 1])
    (product_action_list): TensorShape([10, None, 1])
    (hashed_url_list): TensorShape([10, None, 1])
    (embeddings): TensorShape([10, None, 50])
  )
  (_feature_dtypes): Dict(
    (product_sku_hash_list): tf.int64
    (event_type_list): tf.int64
    (product_action_list): tf.int64
    (hashed_url_list): tf.int64
    (embeddings): tf.float32
  )
),

INFO:tensorflow:Assets written to: trained_model/assets


INFO:tensorflow:Assets written to: trained_model/assets
/usr/local/lib/python3.8/dist-packages/merlin/models/tf/utils/tf_utils.py:101: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  config[key] = tf.keras.utils.serialize_keras_object(maybe_value)
/usr/local/lib/python3.8/dist-packages/merlin/models/tf/core/combinators.py:288: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  config[i] = tf.keras.utils.serialize_keras_object(layer)
/usr/local/lib/python3.8/dist-packages/keras/saving/legacy/saved_model/layer_serialization.py:134: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return serialization.serialize_keras_object(obj)
/usr

## Serving predictions

In [23]:
from merlin.systems.dag.ops.tensorflow import PredictTensorflow
from merlin.systems.dag.ensemble import Ensemble
from merlin.systems.dag.ops.workflow import TransformWorkflow

In [24]:
emb_op = EmbeddingOperator(
    embeddings[:, 1:].astype(np.float32),
    id_lookup_table=embeddings[:, 0].astype(int),
    lookup_key="product_sku_hash_list",
)

In [25]:
inference_operators = wf.input_schema.column_names >> TransformWorkflow(wf) >> emb_op >> PredictTensorflow(model)

  (_feature_shapes): Dict(
    (product_sku_hash_list): TensorShape([10, None, 1])
    (event_type_list): TensorShape([10, None, 1])
    (product_action_list): TensorShape([10, None, 1])
    (hashed_url_list): TensorShape([10, None, 1])
    (embeddings): TensorShape([10, None, 50])
  )
  (_feature_dtypes): Dict(
    (product_sku_hash_list): tf.int64
    (event_type_list): tf.int64
    (product_action_list): tf.int64
    (hashed_url_list): tf.int64
    (embeddings): tf.float32
  )
), because it is not built.


  (_feature_shapes): Dict(
    (product_sku_hash_list): TensorShape([10, None, 1])
    (event_type_list): TensorShape([10, None, 1])
    (product_action_list): TensorShape([10, None, 1])
    (hashed_url_list): TensorShape([10, None, 1])
    (embeddings): TensorShape([10, None, 50])
  )
  (_feature_dtypes): Dict(
    (product_sku_hash_list): tf.int64
    (event_type_list): tf.int64
    (product_action_list): tf.int64
    (hashed_url_list): tf.int64
    (embeddings): tf.float32
  )
), because it is not built.


  (_feature_shapes): Dict(
    (product_sku_hash_list): TensorShape([10, None, 1])
    (event_type_list): TensorShape([10, None, 1])
    (product_action_list): TensorShape([10, None, 1])
    (hashed_url_list): TensorShape([10, None, 1])
    (embeddings): TensorShape([10, None, 50])
  )
  (_feature_dtypes): Dict(
    (product_sku_hash_list): tf.int64
    (event_type_list): tf.int64
    (product_action_list): tf.int64
    (hashed_url_list): tf.int64
    (embeddings): tf.float32
  )
), because it is not built.


  (_feature_shapes): Dict(
    (product_sku_hash_list): TensorShape([10, None, 1])
    (event_type_list): TensorShape([10, None, 1])
    (product_action_list): TensorShape([10, None, 1])
    (hashed_url_list): TensorShape([10, None, 1])
    (embeddings): TensorShape([10, None, 50])
  )
  (_feature_dtypes): Dict(
    (product_sku_hash_list): tf.int64
    (event_type_list): tf.int64
    (product_action_list): tf.int64
    (hashed_url_list): tf.int64
    (embeddings): tf.float32
  )
), because it is not built.


  (_feature_shapes): Dict(
    (product_sku_hash_list): TensorShape([10, None, 1])
    (event_type_list): TensorShape([10, None, 1])
    (product_action_list): TensorShape([10, None, 1])
    (hashed_url_list): TensorShape([10, None, 1])
    (embeddings): TensorShape([10, None, 50])
  )
  (_feature_dtypes): Dict(
    (product_sku_hash_list): tf.int64
    (event_type_list): tf.int64
    (product_action_list): tf.int64
    (hashed_url_list): tf.int64
    (embeddings): tf.float32
  )
), because it is not built.


  (_feature_shapes): Dict(
    (product_sku_hash_list): TensorShape([10, None, 1])
    (event_type_list): TensorShape([10, None, 1])
    (product_action_list): TensorShape([10, None, 1])
    (hashed_url_list): TensorShape([10, None, 1])
    (embeddings): TensorShape([10, None, 50])
  )
  (_feature_dtypes): Dict(
    (product_sku_hash_list): tf.int64
    (event_type_list): tf.int64
    (product_action_list): tf.int64
    (hashed_url_list): tf.int64
    (embeddings): tf.float32
  )
), because it is not built.


INFO:tensorflow:Assets written to: /tmp/tmpw5gia_nw/model.savedmodel/assets


INFO:tensorflow:Assets written to: /tmp/tmpw5gia_nw/model.savedmodel/assets
/usr/local/lib/python3.8/dist-packages/merlin/models/tf/utils/tf_utils.py:101: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  config[key] = tf.keras.utils.serialize_keras_object(maybe_value)
/usr/local/lib/python3.8/dist-packages/merlin/models/tf/core/combinators.py:288: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  config[i] = tf.keras.utils.serialize_keras_object(layer)
/usr/local/lib/python3.8/dist-packages/keras/saving/legacy/saved_model/layer_serialization.py:134: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return serialization.serialize_ke

/usr/local/lib/python3.8/dist-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer TruncatedNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


In [29]:
!rm -rf /workspace/data/ensemble

In [30]:
ensemble = Ensemble(inference_operators, wf.input_schema)
ensemble.export(os.path.join(OUTPUT_DATA_DIR, 'ensemble'));

  (_feature_shapes): Dict(
    (product_sku_hash_list): TensorShape([10, None, 1])
    (event_type_list): TensorShape([10, None, 1])
    (product_action_list): TensorShape([10, None, 1])
    (hashed_url_list): TensorShape([10, None, 1])
    (embeddings): TensorShape([10, None, 50])
  )
  (_feature_dtypes): Dict(
    (product_sku_hash_list): tf.int64
    (event_type_list): tf.int64
    (product_action_list): tf.int64
    (hashed_url_list): tf.int64
    (embeddings): tf.float32
  )
), because it is not built.


  (_feature_shapes): Dict(
    (product_sku_hash_list): TensorShape([10, None, 1])
    (event_type_list): TensorShape([10, None, 1])
    (product_action_list): TensorShape([10, None, 1])
    (hashed_url_list): TensorShape([10, None, 1])
    (embeddings): TensorShape([10, None, 50])
  )
  (_feature_dtypes): Dict(
    (product_sku_hash_list): tf.int64
    (event_type_list): tf.int64
    (product_action_list): tf.int64
    (hashed_url_list): tf.int64
    (embeddings): tf.float32
  )
), because it is not built.


  (_feature_shapes): Dict(
    (product_sku_hash_list): TensorShape([10, None, 1])
    (event_type_list): TensorShape([10, None, 1])
    (product_action_list): TensorShape([10, None, 1])
    (hashed_url_list): TensorShape([10, None, 1])
    (embeddings): TensorShape([10, None, 50])
  )
  (_feature_dtypes): Dict(
    (product_sku_hash_list): tf.int64
    (event_type_list): tf.int64
    (product_action_list): tf.int64
    (hashed_url_list): tf.int64
    (embeddings): tf.float32
  )
), because it is not built.


  (_feature_shapes): Dict(
    (product_sku_hash_list): TensorShape([10, None, 1])
    (event_type_list): TensorShape([10, None, 1])
    (product_action_list): TensorShape([10, None, 1])
    (hashed_url_list): TensorShape([10, None, 1])
    (embeddings): TensorShape([10, None, 50])
  )
  (_feature_dtypes): Dict(
    (product_sku_hash_list): tf.int64
    (event_type_list): tf.int64
    (product_action_list): tf.int64
    (hashed_url_list): tf.int64
    (embeddings): tf.float32
  )
), because it is not built.


  (_feature_shapes): Dict(
    (product_sku_hash_list): TensorShape([10, None, 1])
    (event_type_list): TensorShape([10, None, 1])
    (product_action_list): TensorShape([10, None, 1])
    (hashed_url_list): TensorShape([10, None, 1])
    (embeddings): TensorShape([10, None, 50])
  )
  (_feature_dtypes): Dict(
    (product_sku_hash_list): tf.int64
    (event_type_list): tf.int64
    (product_action_list): tf.int64
    (hashed_url_list): tf.int64
    (embeddings): tf.float32
  )
), because it is not built.


  (_feature_shapes): Dict(
    (product_sku_hash_list): TensorShape([10, None, 1])
    (event_type_list): TensorShape([10, None, 1])
    (product_action_list): TensorShape([10, None, 1])
    (hashed_url_list): TensorShape([10, None, 1])
    (embeddings): TensorShape([10, None, 50])
  )
  (_feature_dtypes): Dict(
    (product_sku_hash_list): tf.int64
    (event_type_list): tf.int64
    (product_action_list): tf.int64
    (hashed_url_list): tf.int64
    (embeddings): tf.float32
  )
), because it is not built.


INFO:tensorflow:Assets written to: /workspace/data/ensemble/1_predicttensorflowtriton/1/model.savedmodel/assets


INFO:tensorflow:Assets written to: /workspace/data/ensemble/1_predicttensorflowtriton/1/model.savedmodel/assets
/usr/local/lib/python3.8/dist-packages/merlin/models/tf/utils/tf_utils.py:101: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  config[key] = tf.keras.utils.serialize_keras_object(maybe_value)
/usr/local/lib/python3.8/dist-packages/merlin/models/tf/core/combinators.py:288: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  config[i] = tf.keras.utils.serialize_keras_object(layer)
/usr/local/lib/python3.8/dist-packages/keras/saving/legacy/saved_model/layer_serialization.py:134: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.

In [ ]:
# start the triton server
# tritonserver --model-repository=/workspace/data/ensemble
# the error should appear in the logs

Let's attempt prediction on a couple of rows from our train set, just for demonstration purposes.

In [ ]:
from merlin.systems.triton import convert_df_to_triton_input

inputs = convert_df_to_triton_input(wf.input_schema, train.head())

In [ ]:
import tritonclient.grpc as grpcclient

with grpcclient.InferenceServerClient("localhost:8001") as client:
    response = client.infer('executor_model', inputs)

In [ ]:
    embeddings = np.load(npy_path)
    # second workflow that categorifies the embedding table data
    df = make_df({"string_id": np.random.choice(string_ids, 30)})
    graph2 = ["string_id"] >> cat_op
    train_res = Workflow(graph2).transform(Dataset(df, cpu=(cpu is not None)))

    data_loader = Loader(
        train_res,
        batch_size=1,
        transforms=[
            EmbeddingOperator(
                embeddings[:, 1:],
                id_lookup_table=embeddings[:, 0].astype(int),
                lookup_key="string_id",
            )
        ],
        shuffle=False,
        device=cpu,
    )
    origin_df = train_res.to_ddf().merge(emb_res.to_ddf(), on="string_id", how="left").compute()
    for idx, batch in enumerate(data_loader):
        batch
        b_df = batch[0].to_df()
        org_df = origin_df.iloc[idx]
        if not cpu:
            assert (b_df["string_id"].to_numpy() == org_df["string_id"].to_numpy()).all()
            assert (b_df["embeddings"].list.leaves == org_df["embeddings"].list.leaves).all()
        else:
            assert (b_df["string_id"].values == org_df["string_id"]).all()
            assert b_df["embeddings"].values[0] == org_df["embeddings"].tolist()

In [ ]:
import cudf
import numpy as np

In [ ]:
df = cudf.DataFrame(data={'id': [1,2,3], 'val': [0, np.nan, 10], 'another_col': ['a', 'b', 'c']})

In [ ]:
df.val[df.val.isna()]

In [ ]:
df.val[~df.val.isna()]

In [ ]:
ds = nvt.Dataset(df)

out = ['val'] >> nvt.ops.Filter(f=lambda col: ~col.isna())

wf = nvt.Workflow(out)
wf.fit_transform(ds).compute()

In [ ]:
out = ['id', 'val'] >> nvt.ops.Filter(f=lambda df: ~df['val'].isna())

wf = nvt.Workflow(out)
wf.fit_transform(ds).compute()

In [ ]:
out = ['id', 'val'] >> nvt.ops.Filter(f=lambda df: ~df['val'].isna())

wf = nvt.Workflow(out + ['another_col'])
wf.fit_transform(ds).compute()

In [ ]:
train.head()

In [ ]:
skus.to_npy('embeddings.npy')

In [ ]:
out = ['product_sku_hash', 'category_hash'] >> nvt.ops.Categorify() >> nvt.ops.TagAsItemID()
out += ['description_vector'] >> nvt.ops.TagAsItemFeatures()
out += ['price_bucket'] >> nvt.ops.NormalizeMinMax()

wf = nvt.Workflow(out)
skus = wf.fit_transform(skus)

In [ ]:
train.head()

In [ ]:
skus.head()

In [111]:
df = cudf.DataFrame(data={'id': [1,2,3], 'label': [1,2,1]})
ds = nvt.Dataset(df)

out = ['label'] >> nvt.ops.AddMetadata(Tags.TARGET)

wf = nvt.Workflow(out + ['id'])

ds_out = wf.fit_transform(ds)

loader = Loader(
    ds_out,
    batch_size=1,
)

loader.peek()

To use synthetically generated data, uncomment the following cell:

In [19]:
%%bash

cd /workspace && pip install . 

Processing /workspace
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'


  Created wheel for merlin-models: filename=merlin_models-23.5.dev0+63.g6d83a8b1.dirty-py3-none-any.whl size=384113 sha256=743286edc1e20e76705ea7980a6cbe6f45accd6b753f72a0815da260a3102641
  Stored in directory: /tmp/pip-ephem-wheel-cache-myac_4jv/wheels/59/14/70/d94958f41745fe226f3bc60bb3cabbbc8a98e4d6679e91038a
Successfully built merlin-models


ERROR: transformers4rec 23.5.0+9.gf4946bfa requires torchmetrics>=0.10.0, which is not installed.


  Attempting uninstall: merlin-models
    Found existing installation: merlin-models 23.5.dev0+57.gad9fa2bb.dirty
    Uninstalling merlin-models-23.5.dev0+57.gad9fa2bb.dirty:
      Successfully uninstalled merlin-models-23.5.dev0+57.gad9fa2bb.dirty


In [3]:
from merlin.datasets.synthetic import KNOWN_DATASETS

In [4]:
KNOWN_DATASETS

{'e-commerce': PosixPath('/usr/local/lib/python3.8/dist-packages/merlin/datasets/ecommerce/small'),
 'e-commerce-large': PosixPath('/usr/local/lib/python3.8/dist-packages/merlin/datasets/ecommerce/large'),
 'music-streaming': PosixPath('/usr/local/lib/python3.8/dist-packages/merlin/datasets/entertainment/music_streaming'),
 'social': PosixPath('/usr/local/lib/python3.8/dist-packages/merlin/datasets/social'),
 'testing': PosixPath('/usr/local/lib/python3.8/dist-packages/merlin/datasets/testing'),
 'sequence-testing': PosixPath('/usr/local/lib/python3.8/dist-packages/merlin/datasets/testing/sequence_testing'),
 'movielens-25m': PosixPath('/usr/local/lib/python3.8/dist-packages/merlin/datasets/entertainment/movielens/25m'),
 'movielens-1m': PosixPath('/usr/local/lib/python3.8/dist-packages/merlin/datasets/entertainment/movielens/1m'),
 'movielens-1m-raw-ratings': PosixPath('/usr/local/lib/python3.8/dist-packages/merlin/datasets/entertainment/movielens/1m-raw/ratings'),
 'movielens-100k': 

,session_id_hash,event_type,product_action,product_sku_hash,hashed_url,server_timestamp_epoch_ms
0,50,0,1,399,949,0.578108
1,8,0,3,505,818,0.471705
2,16,1,0,423,771,0.613716
3,87,1,2,791,49,0.503518
4,378,1,3,424,166,0.864198


,product_sku_hash,description_vector,category_hash,price_bucket
0,35,"[0.28320169822835645, 0.28876255653408484, 0.3...",114,0.148844
1,11,"[0.0498882587601161, 0.4050611778162572, 0.489...",78,0.983131
2,12,"[-0.20383781352758598, 0.2821339201063496, -0....",78,0.268082
3,14,"[0.08096254937074815, 0.5582722991824396, 0.22...",156,0.310764
4,49,"[-0.29878517778423397, -0.3313343019075635, -0...",121,0.097739
